In [124]:
import pandas as pd

In [125]:
#Group 1
tournaments = pd.read_csv("tournaments.csv")
confederations = pd.read_csv("confederations.csv")
teams = pd.read_csv("teams.csv")
players = pd.read_csv("players.csv")
managers = pd.read_csv("managers.csv")
referees = pd.read_csv("referees.csv")
stadiums = pd.read_csv("stadiums.csv")
matches = pd.read_csv("matches.csv")
awards= pd.read_csv("awards.csv")

In [126]:
#Group 2
qualified_teams = pd.read_csv("qualified_teams.csv")
squads = pd.read_csv("squads.csv")
manager_appointment = pd.read_csv("manager_appointments.csv")
refree_appointments = pd.read_csv("referee_appointments.csv")

In [127]:
#Group 3
team_apperance = pd.read_csv("team_appearances.csv")
player_matches = pd.read_csv("player_appearances.csv")
manager_apperance = pd.read_csv("manager_appearances.csv")
refree_appearance = pd.read_csv("referee_appearances.csv")

In [128]:
#Group 4
goals = pd.read_csv("goals.csv")
penalty_kicks = pd.read_csv("penalty_kicks.csv")
booking = pd.read_csv("bookings.csv")
substitutions = pd.read_csv("substitutions.csv")

In [129]:
#Group 5
host_countries = pd.read_csv("host_countries.csv")
tournament_stages = pd.read_csv("tournament_stages.csv")
groups = pd.read_csv("groups.csv")
group_standing = pd.read_csv("group_standings.csv")
tournament_standings = pd.read_csv("tournament_standings.csv")
award_winners = pd.read_csv("award_winners.csv")


UPSET AND SURPRISES - FINAL


In [130]:
import pandas as pd

# Load datasets
players = pd.read_csv('players.csv')  # Replace with the actual file path
tournaments = pd.read_csv('tournaments.csv')
matches = pd.read_csv('matches.csv')
qualified_teams = pd.read_csv('qualified_teams.csv')

# Print the first few rows of each dataset to verify
print("Players dataset:")
print(players.head())

print("\nTournaments dataset:")
print(tournaments.head())

print("\nMatches dataset:")
print(matches.head())

print("\nQualified Teams dataset:")
print(qualified_teams.head())


Players dataset:
   key_id player_id family_name   given_name  birth_date  female  goal_keeper  \
0       1   P-35894     A'Court         Alan  1934-09-30       0            0   
1       2   P-29915     Aarønes  Ann Kristin  1973-01-19       1            0   
2       3   P-03484    Aaronson      Brenden  2000-10-22       0            0   
3       4   P-04189   Abadzhiev       Stefan  1934-07-03       0            0   
4       5   P-03523       Abalo    Jean-Paul  1975-06-26       0            0   

   defender  midfielder  forward  count_tournaments list_tournaments  \
0         0           0        1                  1             1958   
1         0           1        1                  2       1995, 1999   
2         0           0        1                  1             2022   
3         0           0        1                  1             1966   
4         1           0        0                  1             2006   

                               player_wikipedia_link  
0       

In [131]:
# Define the performance mapping
performance_mapping = {
    'final': 1,
    'third-place match': 2,
    'semi-finals': 3,
    'quarter-finals': 4,
    'quarter-final': 4,  # Handle duplicate naming
    'round of 16': 5,
    'second group stage': 6,
    'final round': 7,
    'group stage': 8
}

# Map performance to numerical values
qualified_teams['performance_numeric'] = qualified_teams['performance'].map(performance_mapping)

# Check the updated Qualified Teams dataset
print("Updated Qualified Teams dataset:")
print(qualified_teams.head())


Updated Qualified Teams dataset:
   key_id tournament_id            tournament_name team_id  team_name  \
0       1       WC-1930  1930 FIFA Men's World Cup    T-03  Argentina   
1       2       WC-1930  1930 FIFA Men's World Cup    T-06    Belgium   
2       3       WC-1930  1930 FIFA Men's World Cup    T-07    Bolivia   
3       4       WC-1930  1930 FIFA Men's World Cup    T-09     Brazil   
4       5       WC-1930  1930 FIFA Men's World Cup    T-13      Chile   

  team_code  count_matches  performance  performance_numeric  
0       ARG              5        final                    1  
1       BEL              2  group stage                    8  
2       BOL              2  group stage                    8  
3       BRA              2  group stage                    8  
4       CHL              3  group stage                    8  


In [132]:
# Merge home team performance into matches
matches = pd.merge(
    matches,
    qualified_teams[['tournament_id', 'team_id', 'performance_numeric']],
    left_on=['tournament_id', 'home_team_id'],
    right_on=['tournament_id', 'team_id'],
    how='left'
).rename(columns={'performance_numeric': 'home_team_performance'})

# Drop the duplicate team_id column to avoid conflicts
matches = matches.drop(columns=['team_id'])

# Merge away team performance into matches
matches = pd.merge(
    matches,
    qualified_teams[['tournament_id', 'team_id', 'performance_numeric']],
    left_on=['tournament_id', 'away_team_id'],
    right_on=['tournament_id', 'team_id'],
    how='left'
).rename(columns={'performance_numeric': 'away_team_performance'})

# Drop the duplicate team_id column again
matches = matches.drop(columns=['team_id'])

# Check the updated Matches dataset
print("Updated Matches dataset:")
print(matches[['match_id', 'home_team_id', 'home_team_performance', 'away_team_id', 'away_team_performance']].head())


Updated Matches dataset:
    match_id home_team_id  home_team_performance away_team_id  \
0  M-1930-01         T-30                      8         T-46   
1  M-1930-02         T-83                      3         T-06   
2  M-1930-03         T-87                      3         T-09   
3  M-1930-04         T-61                      8         T-56   
4  M-1930-05         T-03                      1         T-30   

   away_team_performance  
0                      8  
1                      8  
2                      8  
3                      8  
4                      8  


In [133]:
# Step 1: Calculate performance difference
matches['performance_diff'] = matches['home_team_performance'] - matches['away_team_performance']

# Step 2: Classify matches based on outcomes
def classify_match(row):
    if row['home_team_performance'] < row['away_team_performance'] and row['home_team_win'] == 1:
        return 'Upset'
    elif row['home_team_performance'] > row['away_team_performance'] and row['away_team_win'] == 1:
        return 'Upset'
    elif abs(row['performance_diff']) <= 1 and row['draw'] == 1:
        return 'Surprise'
    else:
        return 'Expected'

matches['classification'] = matches.apply(classify_match, axis=1)

# Step 3: Filter for upsets and surprises
upsets_and_surprises = matches[matches['classification'].isin(['Upset', 'Surprise'])]

# Check the resulting dataset
print("Upsets and Surprises:")
print(upsets_and_surprises[['match_id', 'home_team_id', 'away_team_id', 'classification']].head())

# Save the upsets and surprises dataset
upsets_and_surprises.to_csv('upsets_and_surprises.csv', index=False)


Upsets and Surprises:
    match_id home_team_id away_team_id classification
1  M-1930-02         T-83         T-06          Upset
2  M-1930-03         T-87         T-09          Upset
4  M-1930-05         T-03         T-30          Upset
6  M-1930-07         T-87         T-07          Upset
7  M-1930-08         T-83         T-55          Upset


Golden Boot analysis - final

In [2]:
import pandas as pd

# Load datasets
award_winners = pd.read_csv('award_winners.csv')
goals = pd.read_csv('goals.csv')
players = pd.read_csv('players.csv')

# Step 1: Filter for relevant awards
# Focus on 'Golden Boot' and other top scorer-related awards
golden_boot_winners = award_winners[award_winners['award_name'] == 'Golden Boot']

# Step 2: Extract relevant columns for awards
golden_boot_winners = golden_boot_winners[['tournament_id', 'tournament_name', 'player_id', 
                                           'family_name', 'given_name', 'team_name', 'team_code']]

# Step 3: Derive goals scored by players in each tournament
goals_summary = goals.groupby(['tournament_id', 'player_id']).agg({'goal_id': 'count'}).reset_index()
goals_summary.rename(columns={'goal_id': 'goals_scored'}, inplace=True)

# Step 4: Add player position information
def get_position(row):
    if row['goal_keeper'] == 1:
        return "Goalkeeper"
    elif row['defender'] == 1:
        return "Defender"
    elif row['midfielder'] == 1:
        return "Midfielder"
    elif row['forward'] == 1:
        return "Forward"
    else:
        return "Unknown"

players['position'] = players.apply(get_position, axis=1)

# Step 5: Merge datasets for comprehensive analysis
# Merge goals with player details
golden_race_data = pd.merge(goals_summary, 
                            players[['player_id', 'family_name', 'given_name', 'position']], 
                            on='player_id', how='left')

# Add tournament name from Golden Boot winners
golden_race_data = pd.merge(golden_race_data, 
                            golden_boot_winners[['tournament_id', 'tournament_name']], 
                            on='tournament_id', how='left')

# Add team name and code from awards dataset
golden_race_data = pd.merge(golden_race_data, 
                            golden_boot_winners[['player_id', 'team_name', 'team_code']], 
                            on='player_id', how='left')

# Step 6: Add award details
# Merge all awards for players who won any awards (including Golden Boot)
golden_race_data = pd.merge(golden_race_data, 
                            award_winners[['player_id', 'award_name']], 
                            on='player_id', how='left')

# Step 7: Perform advanced calculations
# Add rank by goals scored per tournament
golden_race_data['rank'] = golden_race_data.groupby('tournament_id')['goals_scored'].rank(ascending=False, method='min')

# Identify multi-tournament players
multi_tournament_players = golden_race_data.groupby('player_id').size().reset_index(name='tournaments_participated')
multi_tournament_players = multi_tournament_players[multi_tournament_players['tournaments_participated'] > 1]

# Merge with main dataset
golden_race_data = pd.merge(golden_race_data, multi_tournament_players[['player_id', 'tournaments_participated']], 
                            on='player_id', how='left')

# Calculate average goals scored per tournament
avg_goals_per_tournament = golden_race_data.groupby('player_id')['goals_scored'].mean().reset_index()
avg_goals_per_tournament.rename(columns={'goals_scored': 'avg_goals_per_tournament'}, inplace=True)
golden_race_data = pd.merge(golden_race_data, avg_goals_per_tournament, on='player_id', how='left')

# Step 8: Save the enriched dataset
golden_race_data.to_csv('Golden boot race.csv', index=False)

# Display key insights
print("Golden Boot Race Enhanced Insights:")
print(golden_race_data.head())
print(f"Golden Boot Race Dataset shape: {golden_race_data.shape}")

# Optional: Suggestions for advanced Tableau/Visualization
# - Award Categories per Player: Analyze the distribution of awards across players.
# - Goals Trend: Show goals scored by top players across multiple tournaments.
# - Multi-Tournament Legends: Highlight players with multiple tournament participations and their rankings.
# - Position-Based Performance: Heatmap of goals scored by positions per tournament.


Golden Boot Race Enhanced Insights:
  tournament_id player_id  goals_scored  family_name  given_name position  \
0       WC-1930   P-05470             1      Laurent      Lucien  Forward   
1       WC-1930   P-06201             1  Vargas Peña        Luis  Forward   
2       WC-1930   P-06424             1        Brown         Jim  Forward   
3       WC-1930   P-08459             2     Subiabre   Guillermo  Forward   
4       WC-1930   P-18615             1      Stanciu  Constantin  Forward   

             tournament_name team_name team_code award_name  rank  \
0  1930 FIFA Men's World Cup       NaN       NaN        NaN  19.0   
1  1930 FIFA Men's World Cup       NaN       NaN        NaN  19.0   
2  1930 FIFA Men's World Cup       NaN       NaN        NaN  19.0   
3  1930 FIFA Men's World Cup       NaN       NaN        NaN   9.0   
4  1930 FIFA Men's World Cup       NaN       NaN        NaN  19.0   

   tournaments_participated  avg_goals_per_tournament  
0                       NaN   

World cup legends - Final

In [9]:
import pandas as pd

# Load the datasets
players = pd.read_csv('players.csv')
managers = pd.read_csv('managers.csv')
referees = pd.read_csv('referees.csv')
award_winners = pd.read_csv('award_winners.csv')
player_appearances = pd.read_csv('player_appearances.csv')
manager_appearances = pd.read_csv('manager_appearances.csv')
referee_appearances = pd.read_csv('referee_appearances.csv')
goals = pd.read_csv('goals.csv')
bookings = pd.read_csv('bookings.csv')

# Step 1: Add Derived Statistics for Players
# Rename tournament_id in players_appearances for clarity
player_appearances.rename(columns={'tournament_id': 'tournament_id_x'}, inplace=True)

# Merge players with appearances
players_appearances = pd.merge(players, player_appearances, on='player_id', how='left')

# Add goals data
players_with_goals = pd.merge(players_appearances, goals[['player_id', 'match_id', 'goal_id', 'tournament_id']], on='player_id', how='left')

# Combine tournament_id_x and tournament_id from goals
players_with_goals['tournament_id'] = players_with_goals['tournament_id_x'].fillna(players_with_goals['tournament_id'])

# Add goals scored column
players_with_goals['goals_scored'] = players_with_goals['goal_id'].notna().astype(int)

# Rename match_id_x to match_id for consistency
players_with_goals.rename(columns={'match_id_x': 'match_id'}, inplace=True)

# Merge with bookings data
players_with_bookings = pd.merge(
    players_with_goals,
    bookings[['player_id', 'match_id', 'yellow_card', 'red_card']],
    on=['player_id', 'match_id'],
    how='left'
)


# Aggregate statistics
player_stats = players_with_bookings.groupby('player_id').agg({
    'match_id': 'nunique',  # Matches played
    'goals_scored': 'sum',  # Total goals
    'yellow_card': 'sum',   # Total yellow cards
    'red_card': 'sum',      # Total red cards
    'tournament_id': 'nunique',  # Tournaments participated
    'team_id': 'first'      # Include team_id
}).reset_index()

# Rename columns
player_stats.rename(columns={
    'match_id': 'matches_played',
    'yellow_card': 'total_yellow_cards',
    'red_card': 'total_red_cards',
    'tournament_id': 'tournaments_participated'
}, inplace=True)

# Add player names and roles
player_stats = pd.merge(player_stats, 
                        players[['player_id', 'family_name', 'given_name', 'forward', 'midfielder', 'defender', 'goal_keeper']], 
                        on='player_id', how='left')
player_stats['role'] = player_stats.apply(lambda row: 'Forward' if row['forward'] == 1 else 
                                          ('Midfielder' if row['midfielder'] == 1 else
                                           ('Defender' if row['defender'] == 1 else
                                            ('Goalkeeper' if row['goal_keeper'] == 1 else 'Unknown'))), axis=1)

# Step 2: Add Derived Statistics for Managers
# Merge managers with appearances
managers_appearances = pd.merge(managers, manager_appearances, on='manager_id', how='left')

# Aggregate manager statistics
manager_stats = managers_appearances.groupby('manager_id').agg({
    'match_id': 'nunique',  # Matches managed
    'tournament_id': 'nunique'  # Tournaments participated
}).reset_index()

# Rename columns
manager_stats.rename(columns={
    'match_id': 'matches_managed',
    'tournament_id': 'tournaments_participated'
}, inplace=True)

# Add manager names and roles
manager_stats = pd.merge(manager_stats, 
                         managers[['manager_id', 'family_name', 'given_name']], 
                         on='manager_id', how='left')
manager_stats['role'] = 'Manager'

# Step 3: Add Derived Statistics for Referees
# Merge referees with appearances
referees_appearances = pd.merge(referees, referee_appearances, on='referee_id', how='left')

# Aggregate referee statistics
referee_stats = referees_appearances.groupby('referee_id').agg({
    'match_id': 'nunique',  # Matches officiated
    'tournament_id': 'nunique'  # Tournaments participated
}).reset_index()

# Rename columns
referee_stats.rename(columns={
    'match_id': 'matches_officiated',
    'tournament_id': 'tournaments_participated'
}, inplace=True)

# Add referee names and roles
referee_stats = pd.merge(referee_stats, 
                         referees[['referee_id', 'family_name', 'given_name']], 
                         on='referee_id', how='left')
referee_stats['role'] = 'Referee'

# Step 4: Standardize and Combine
# Standardize column names for merging
player_stats.rename(columns={'player_id': 'id'}, inplace=True)
manager_stats.rename(columns={'manager_id': 'id'}, inplace=True)
referee_stats.rename(columns={'referee_id': 'id'}, inplace=True)

# Combine all profiles
world_cup_legends = pd.concat([player_stats[['id', 'matches_played', 'goals_scored', 
                                             'total_yellow_cards', 'total_red_cards', 
                                             'tournaments_participated', 'team_id', 
                                             'family_name', 'given_name', 'role']], 
                               manager_stats[['id', 'matches_managed', 
                                              'tournaments_participated', 'family_name', 
                                              'given_name', 'role']], 
                               referee_stats[['id', 'matches_officiated', 
                                              'tournaments_participated', 'family_name', 
                                              'given_name', 'role']]], 
                              ignore_index=True)

# Add Awards Insights
awards_summary = pd.merge(award_winners[['player_id', 'award_name']],
                          players[['player_id', 'family_name', 'given_name']], 
                          on='player_id', how='inner')
world_cup_legends = pd.merge(world_cup_legends, 
                             awards_summary[['player_id', 'award_name']], 
                             left_on='id', right_on='player_id', how='left')
world_cup_legends.drop('player_id', axis=1, inplace=True)

# Save the final dataset
world_cup_legends.to_csv('world_cup_legends_with_team_tournament.csv', index=False)

# Display the final dataset
print(world_cup_legends.head())
print(f"World Cup Legends Profile dataset shape: {world_cup_legends.shape}")


        id  matches_played  goals_scored  total_yellow_cards  total_red_cards  \
0  P-00020             3.0           0.0                 1.0              0.0   
1  P-00034             2.0           0.0                 0.0              0.0   
2  P-00036             0.0           0.0                 0.0              0.0   
3  P-00042             0.0           1.0                 0.0              0.0   
4  P-00052             3.0           0.0                 0.0              0.0   

   tournaments_participated team_id  family_name      given_name      role  \
0                         1    T-12       Hermus          Randee  Defender   
1                         1    T-73      Alfonso  not applicable   Forward   
2                         0    None        Strik           Pleun  Defender   
3                         1    None  Milutinović           Miloš   Forward   
4                         1    T-59        Ró-Ró  not applicable  Defender   

   matches_managed  matches_officiated award

Road to victory - Final

In [136]:
import pandas as pd

# Load datasets
matches = pd.read_csv('matches.csv')
tournaments = pd.read_csv('tournaments.csv')

# Display dataset previews
print("Matches dataset preview:")
print(matches.head())

print("\nTournaments dataset preview:")
print(tournaments.head())

# Step 1: Filter matches involving the winner country
# Merge tournaments with matches to link the winner to matches
winner_matches = pd.merge(matches, tournaments[['tournament_id', 'winner']], on='tournament_id', how='inner')

# Filter for matches where the winner is either the home team or away team
winner_matches = winner_matches[
    (winner_matches['home_team_name'] == winner_matches['winner']) |
    (winner_matches['away_team_name'] == winner_matches['winner'])
]

# Display the filtered matches
print("Winner Matches:")
print(winner_matches[['tournament_id', 'tournament_name', 'match_id', 'stage_name', 
                      'match_date', 'home_team_name', 'away_team_name', 
                      'home_team_score', 'away_team_score', 'winner']].head())

# Step 2: Add "result" column for match outcomes
def get_result(row):
    if row['home_team_name'] == row['winner'] and row['home_team_score'] > row['away_team_score']:
        return 'Win'
    elif row['away_team_name'] == row['winner'] and row['away_team_score'] > row['home_team_score']:
        return 'Win'
    elif row['home_team_score'] == row['away_team_score']:
        return 'Draw'
    else:
        return 'Loss'

winner_matches['result'] = winner_matches.apply(get_result, axis=1)

# Display results
print("Winner Matches with Results:")
print(winner_matches[['tournament_id', 'tournament_name', 'match_id', 'stage_name', 
                      'match_date', 'home_team_name', 'away_team_name', 
                      'home_team_score', 'away_team_score', 'winner', 'result']].head())

# Step 3: Add performance metrics
winner_matches['goal_difference'] = abs(winner_matches['home_team_score'] - winner_matches['away_team_score'])
winner_matches['dominant_win'] = winner_matches['goal_difference'] >= 3
winner_matches['close_match'] = winner_matches['goal_difference'] == 1

# Step 4: Group and aggregate data for deeper insights
performance_summary = winner_matches.groupby('tournament_id').agg({
    'match_id': 'count',  # Total matches played
    'goal_difference': 'mean',  # Average goal difference
    'dominant_win': 'sum',  # Number of dominant wins
    'close_match': 'sum',  # Number of close matches
}).reset_index()
performance_summary.rename(columns={'match_id': 'total_matches'}, inplace=True)

# Display summary
print("\nPerformance Summary:")
print(performance_summary.head())

# Step 5: Analyze stage performance
stage_performance = winner_matches.groupby(['tournament_id', 'stage_name']).agg({
    'match_id': 'count',
    'result': lambda x: (x == 'Win').sum()
}).reset_index()
stage_performance.rename(columns={'match_id': 'matches_played', 'result': 'wins'}, inplace=True)

# Display stage performance
print("\nStage Performance:")
print(stage_performance.head())

# Step 6: Analyze home vs. away performance
winner_matches['is_home'] = winner_matches['home_team_name'] == winner_matches['winner']
home_away_performance = winner_matches.groupby('is_home').agg({
    'match_id': 'count',
    'goal_difference': 'mean',
    'result': lambda x: (x == 'Win').sum()
}).reset_index()
home_away_performance.rename(columns={'match_id': 'matches', 'goal_difference': 'avg_goal_difference', 'result': 'wins'}, inplace=True)

# Display home vs. away performance
print("\nHome vs. Away Performance:")
print(home_away_performance)

# Step 7: Finalize the "Road to Victory" storyline dataset
winner_journey = winner_matches.sort_values(by=['tournament_id', 'match_date'])
winner_journey = winner_journey[['tournament_id', 'tournament_name', 'match_id', 'stage_name', 
                                 'match_date', 'home_team_name', 'away_team_name', 
                                 'home_team_score', 'away_team_score', 'goal_difference', 
                                 'result', 'dominant_win', 'close_match']]

# Save the journey dataset for visualization
winner_journey.to_csv('Road_to_victory_detailed.csv', index=False)

# Display the final dataset
print("\nFinal Winner Journey Dataset:")
print(winner_journey.head())
print(f"Final dataset shape: {winner_journey.shape}")


Matches dataset preview:
   key_id tournament_id            tournament_name   match_id  \
0       1       WC-1930  1930 FIFA Men's World Cup  M-1930-01   
1       2       WC-1930  1930 FIFA Men's World Cup  M-1930-02   
2       3       WC-1930  1930 FIFA Men's World Cup  M-1930-03   
3       4       WC-1930  1930 FIFA Men's World Cup  M-1930-04   
4       5       WC-1930  1930 FIFA Men's World Cup  M-1930-05   

                 match_name   stage_name group_name  group_stage  \
0          France vs Mexico  group stage    Group 1            1   
1  United States vs Belgium  group stage    Group 4            1   
2      Yugoslavia vs Brazil  group stage    Group 2            1   
3           Romania vs Peru  group stage    Group 3            1   
4       Argentina vs France  group stage    Group 1            1   

   knockout_stage  replayed  ...  away_team_score_margin extra_time  \
0               0         0  ...                      -3          0   
1               0         0  ... 

Global Engagement - final

In [137]:
import pandas as pd

# Load datasets
tournaments = pd.read_csv('tournaments.csv')
matches = pd.read_csv('matches.csv')
qualified_teams = pd.read_csv('qualified_teams.csv')

# Step 1: Aggregate tournaments by host country
host_country_stats = tournaments.groupby('host_country').agg({
    'tournament_id': 'count',       # Number of tournaments hosted
    'count_teams': 'sum',           # Total teams across all tournaments
}).reset_index()
host_country_stats.rename(columns={'tournament_id': 'tournaments_hosted'}, inplace=True)

# Step 2: Add match density per tournament
matches_per_tournament = matches.groupby('tournament_id').size().reset_index(name='total_matches')
host_country_stats = pd.merge(host_country_stats, 
                               tournaments[['tournament_id', 'host_country']].merge(matches_per_tournament, on='tournament_id'), 
                               on='host_country', how='left')
host_country_stats['average_matches_per_tournament'] = host_country_stats['total_matches'] / host_country_stats['tournaments_hosted']

# Step 3: Combine with qualified teams for deeper insights
team_participation = qualified_teams.groupby('tournament_id').size().reset_index(name='total_teams')
team_participation = pd.merge(team_participation, tournaments[['tournament_id', 'host_country']], on='tournament_id', how='left')
host_country_stats = pd.merge(host_country_stats, team_participation.groupby('host_country').agg({'total_teams': 'mean'}).reset_index(), on='host_country', how='left')

# Save for Tableau
host_country_stats.to_csv('Global_Engagement_Analysis.csv', index=False)
print("Global Engagement analysis dataset created.")


Global Engagement analysis dataset created.


Breaking records

In [138]:
# Load datasets
matches = pd.read_csv('matches.csv')
goals = pd.read_csv('goals.csv')

# Step 1: Identify matches with the highest scores
match_scores = matches.copy()
match_scores['total_goals'] = match_scores['home_team_score'] + match_scores['away_team_score']
highest_scoring_matches = match_scores.sort_values(by='total_goals', ascending=False).head(10)

# Step 2: Calculate fastest goals scored
fastest_goals = goals.sort_values(by=['minute_regulation', 'minute_stoppage'], ascending=[True, True]).head(10)
fastest_goals = fastest_goals[['tournament_id', 'player_id', 'family_name', 'given_name', 'minute_regulation', 'minute_stoppage', 'match_id']]

# Step 3: Aggregate team records
team_records = matches.groupby('tournament_id').agg({
    'home_team_score': 'max',  # Highest goals scored in a match by a team
    'away_team_score': 'max',
}).reset_index()
team_records['highest_team_score'] = team_records[['home_team_score', 'away_team_score']].max(axis=1)

# Step 4: Save datasets for Tableau
highest_scoring_matches.to_csv('Highest_Scoring_Matches.csv', index=False)
fastest_goals.to_csv('Fastest_Goals.csv', index=False)
team_records.to_csv('Team_Records.csv', index=False)
print("Breaking Records datasets created.")


Breaking Records datasets created.


Rivalries and Repeat Matches

In [10]:
import pandas as pd

# Load datasets
matches = pd.read_csv('matches.csv')

# Step 1: Identify Repeat Matches
# Group by teams (home and away) to calculate the number of matches played between each pair
repeat_matches = matches.groupby(['home_team_id', 'home_team_name', 'away_team_id', 'away_team_name']).agg({
    'match_id': 'count',             # Total matches played
    'home_team_score': 'sum',        # Total goals scored by home team
    'away_team_score': 'sum',        # Total goals scored by away team
}).reset_index()

# Rename columns for clarity
repeat_matches.rename(columns={
    'match_id': 'total_matches',
    'home_team_score': 'home_goals',
    'away_team_score': 'away_goals'
}, inplace=True)

# Add a "total_goals" column for combined goals in all matches
repeat_matches['total_goals'] = repeat_matches['home_goals'] + repeat_matches['away_goals']

# Add a "goal_difference" column
repeat_matches['goal_difference'] = repeat_matches['home_goals'] - repeat_matches['away_goals']

# Step 2: Historic Rivalries
# Filter pairs that have played more than once
rivalries = repeat_matches[repeat_matches['total_matches'] > 1].copy()

# Add win/loss/draw stats
rivalries['home_wins'] = matches[matches['home_team_id'].isin(rivalries['home_team_id'])].groupby(
    ['home_team_id', 'away_team_id'])['home_team_win'].sum().reset_index(drop=True)
rivalries['away_wins'] = matches[matches['away_team_id'].isin(rivalries['away_team_id'])].groupby(
    ['home_team_id', 'away_team_id'])['away_team_win'].sum().reset_index(drop=True)
rivalries['draws'] = rivalries['total_matches'] - (rivalries['home_wins'] + rivalries['away_wins'])

# Step 3: Add Memorable Matches
# Filter matches dataset for pairs in rivalries
memorable_matches = matches[
    matches.apply(
        lambda x: (x['home_team_id'], x['away_team_id']) in zip(rivalries['home_team_id'], rivalries['away_team_id']) or
                  (x['away_team_id'], x['home_team_id']) in zip(rivalries['home_team_id'], rivalries['away_team_id']),
        axis=1
    )
].copy()

# Step 4: Summarize Rivalry Data for Visualization
rivalries['avg_goals_per_match'] = rivalries['total_goals'] / rivalries['total_matches']

# Include tournament_id and match_id for additional insights
rivalries = pd.merge(rivalries, matches[['home_team_id', 'away_team_id', 'tournament_id']].drop_duplicates(), 
                     on=['home_team_id', 'away_team_id'], how='left')
memorable_matches = memorable_matches[['match_id', 'home_team_id', 'home_team_name', 
                                       'away_team_id', 'away_team_name', 'tournament_id', 
                                       'home_team_score', 'away_team_score']]

# Save datasets for Tableau visualization
rivalries.to_csv('Rivalries_Data.csv', index=False)
memorable_matches.to_csv('Memorable_Matches.csv', index=False)

# Step 5: Prepare Data for Network Graph
network_data = rivalries[['home_team_id', 'away_team_id', 'home_team_name', 'away_team_name', 
                          'total_matches', 'total_goals', 'tournament_id']]
network_data.to_csv('Network_Data.csv', index=False)

# Output
print("Rivalries dataset created:")
print(rivalries.head())

print("\nMemorable Matches dataset created:")
print(memorable_matches.head())

print("\nNetwork Data for Visualization:")
print(network_data.head())


Rivalries dataset created:
  home_team_id home_team_name away_team_id away_team_name  total_matches  \
0         T-03      Argentina         T-06        Belgium              3   
1         T-03      Argentina         T-06        Belgium              3   
2         T-03      Argentina         T-06        Belgium              3   
3         T-03      Argentina         T-09         Brazil              3   
4         T-03      Argentina         T-09         Brazil              3   

   home_goals  away_goals  total_goals  goal_difference  home_wins  away_wins  \
0           3           1            4                2        0.0        2.0   
1           3           1            4                2        0.0        2.0   
2           3           1            4                2        0.0        2.0   
3           2           5            7               -3        2.0        1.0   
4           2           5            7               -3        2.0        1.0   

   draws  avg_goals_per_match

Evolution of Tactics and Strategies

In [140]:
print(matches.columns)
print(team_apperance.columns)
print(tournaments.columns)


Index(['key_id', 'tournament_id', 'tournament_name', 'match_id', 'match_name',
       'stage_name', 'group_name', 'group_stage', 'knockout_stage', 'replayed',
       'replay', 'match_date', 'match_time', 'stadium_id', 'stadium_name',
       'city_name', 'country_name', 'home_team_id', 'home_team_name',
       'home_team_code', 'away_team_id', 'away_team_name', 'away_team_code',
       'score', 'home_team_score', 'away_team_score', 'home_team_score_margin',
       'away_team_score_margin', 'extra_time', 'penalty_shootout',
       'score_penalties', 'home_team_score_penalties',
       'away_team_score_penalties', 'result', 'home_team_win', 'away_team_win',
       'draw'],
      dtype='object')
Index(['key_id', 'tournament_id', 'tournament_name', 'match_id', 'match_name',
       'stage_name', 'group_name', 'group_stage', 'knockout_stage', 'replayed',
       'replay', 'match_date', 'match_time', 'stadium_id', 'stadium_name',
       'city_name', 'country_name', 'team_id', 'team_name', 'team

In [141]:
# Merge matches and team_appearances on 'match_id'
merged_data = pd.merge(
    matches,
    team_apperance,
    on='match_id',
    suffixes=('_match', '_team')
)

# Display the first few rows of the merged dataset
print("Merged Data Preview:")
print(merged_data.head())

# Display column names of the merged dataset
print("Merged Data Columns:", merged_data.columns)


Merged Data Preview:
   key_id_match tournament_id_match      tournament_name_match   match_id  \
0             1             WC-1930  1930 FIFA Men's World Cup  M-1930-01   
1             1             WC-1930  1930 FIFA Men's World Cup  M-1930-01   
2             2             WC-1930  1930 FIFA Men's World Cup  M-1930-02   
3             2             WC-1930  1930 FIFA Men's World Cup  M-1930-02   
4             3             WC-1930  1930 FIFA Men's World Cup  M-1930-03   

           match_name_match stage_name_match group_name_match  \
0          France vs Mexico      group stage          Group 1   
1          France vs Mexico      group stage          Group 1   
2  United States vs Belgium      group stage          Group 4   
3  United States vs Belgium      group stage          Group 4   
4      Yugoslavia vs Brazil      group stage          Group 2   

   group_stage_match  knockout_stage_match  replayed_match  ...  \
0                  1                     0               0

In [142]:
# Merge merged_data with tournaments on 'tournament_id'
final_data = pd.merge(
    merged_data,
    tournaments[['tournament_id', 'year', 'host_country', 'winner']],
    left_on='tournament_id_match',
    right_on='tournament_id',
    how='left'
)

# Drop duplicate columns if necessary
final_data = final_data.loc[:, ~final_data.columns.duplicated()]

# Display the first few rows of the final dataset
print("Final Data Preview:")
print(final_data.head())

# Display column names of the final dataset
print("Final Data Columns:", final_data.columns)


Final Data Preview:
   key_id_match tournament_id_match      tournament_name_match   match_id  \
0             1             WC-1930  1930 FIFA Men's World Cup  M-1930-01   
1             1             WC-1930  1930 FIFA Men's World Cup  M-1930-01   
2             2             WC-1930  1930 FIFA Men's World Cup  M-1930-02   
3             2             WC-1930  1930 FIFA Men's World Cup  M-1930-02   
4             3             WC-1930  1930 FIFA Men's World Cup  M-1930-03   

           match_name_match stage_name_match group_name_match  \
0          France vs Mexico      group stage          Group 1   
1          France vs Mexico      group stage          Group 1   
2  United States vs Belgium      group stage          Group 4   
3  United States vs Belgium      group stage          Group 4   
4      Yugoslavia vs Brazil      group stage          Group 2   

   group_stage_match  knockout_stage_match  replayed_match  ...  \
0                  1                     0               0 

In [143]:
# Group by tournament, team, and stage
tactical_analysis = final_data.groupby(
    ['year', 'tournament_name_match', 'stage_name_match', 'team_name']
).agg({
    'goals_for': 'sum',               # Total goals scored
    'goals_against': 'sum',           # Total goals conceded
    'goal_differential': 'mean',      # Average goal differential
    'win': 'sum',                     # Total wins
    'lose': 'sum',                    # Total losses
    'draw_team': 'sum',               # Total draws
    'match_id': 'count'               # Matches played
}).reset_index()

# Calculate derived metrics
tactical_analysis['win_rate'] = (tactical_analysis['win'] / tactical_analysis['match_id']) * 100
tactical_analysis['avg_goals_per_match'] = tactical_analysis['goals_for'] / tactical_analysis['match_id']
tactical_analysis['avg_conceded_per_match'] = tactical_analysis['goals_against'] / tactical_analysis['match_id']

# Preview the dataset
print("Tactical Analysis Dataset:")
print(tactical_analysis.head())

# Save to CSV
tactical_analysis.to_csv('tactical_analysis_dataset.csv', index=False)

print("Tactical analysis dataset saved as 'tactical_analysis_dataset.csv'.")


Tactical Analysis Dataset:
   year      tournament_name_match stage_name_match  team_name  goals_for  \
0  1930  1930 FIFA Men's World Cup            final  Argentina          2   
1  1930  1930 FIFA Men's World Cup            final    Uruguay          4   
2  1930  1930 FIFA Men's World Cup      group stage  Argentina         10   
3  1930  1930 FIFA Men's World Cup      group stage    Belgium          0   
4  1930  1930 FIFA Men's World Cup      group stage    Bolivia          0   

   goals_against  goal_differential  win  lose  draw_team  match_id  win_rate  \
0              4               -2.0    0     1          0         1       0.0   
1              2                2.0    1     0          0         1     100.0   
2              4                2.0    3     0          0         3     100.0   
3              4               -2.0    0     2          0         2       0.0   
4              8               -4.0    0     2          0         2       0.0   

   avg_goals_per_match 

Penalty Shootouts

In [13]:
import pandas as pd

# Load datasets
matches = pd.read_csv('matches.csv')
penalty_kicks = pd.read_csv('penalty_kicks.csv')
tournaments = pd.read_csv('tournaments.csv')

# Step 1: Filter matches with penalty shootouts
# Remove redundant `tournament_name` from matches to avoid conflicts during merge
matches = matches.drop(columns=['tournament_name'])

# Merge matches with tournaments for additional information
penalty_shootout_data = pd.merge(
    matches[matches['penalty_shootout'] == True],  # Filter penalty shootout matches
    tournaments[['tournament_id', 'tournament_name', 'year']],
    on='tournament_id',
    how='left'
)

# Select relevant columns
penalty_shootout_data = penalty_shootout_data[[
    'tournament_id',  # Include tournament_id
    'tournament_name',
    'year',
    'stage_name',
    'match_id',  # Include match_id
    'match_name',
    'home_team_id',  # Include home_team_id
    'home_team_name',
    'away_team_id',  # Include away_team_id
    'away_team_name',
    'home_team_score_penalties',
    'away_team_score_penalties',
    'score_penalties',
    'result'
]]

# Rename columns for clarity
penalty_shootout_data.rename(columns={
    'result': 'match_result'
}, inplace=True)

# Add penalty winner column
penalty_shootout_data['penalty_winner'] = penalty_shootout_data.apply(
    lambda row: row['home_team_name'] if row['home_team_score_penalties'] > row['away_team_score_penalties']
    else row['away_team_name'],
    axis=1
)

# Step 2: Aggregate data for penalty shootout performance by team
penalty_performance = penalty_kicks.groupby(['team_id', 'team_name']).agg({
    'converted': ['sum', 'count']  # Total penalties scored and attempted
}).reset_index()

# Rename columns for clarity
penalty_performance.columns = ['team_id', 'team_name', 'penalties_scored', 'penalties_attempted']

# Calculate penalty conversion rate
penalty_performance['conversion_rate'] = (penalty_performance['penalties_scored'] / penalty_performance['penalties_attempted']) * 100

# Step 3: Merge penalty performance with tournament results
penalty_tournament_data = pd.merge(
    penalty_performance,
    penalty_shootout_data,
    left_on='team_name',
    right_on='home_team_name',
    how='left'
)

# Add additional metrics
penalty_tournament_data['shootout_wins'] = penalty_tournament_data['penalty_winner'] == penalty_tournament_data['team_name']
penalty_tournament_data['shootout_wins'] = penalty_tournament_data['shootout_wins'].astype(int)

# Step 4: Create metrics for Tableau analysis
# Aggregate shootout wins and conversion rates by team
team_penalty_stats = penalty_tournament_data.groupby(['team_id', 'team_name']).agg({
    'penalties_scored': 'sum',
    'penalties_attempted': 'sum',
    'conversion_rate': 'mean',
    'shootout_wins': 'sum',
    'tournament_id': 'nunique',  # Count unique tournaments
    'match_id': 'nunique'       # Count unique matches
}).reset_index()

# Step 5: Save datasets for Tableau
penalty_shootout_data.to_csv('penalty_shootout_analysis.csv', index=False)
penalty_performance.to_csv('penalty_performance.csv', index=False)
team_penalty_stats.to_csv('team_penalty_stats.csv', index=False)

# Final Output
print("Penalty Shootout Data:")
print(penalty_shootout_data.head())

print("\nPenalty Performance Data:")
print(penalty_performance.head())

print("\nTeam Penalty Stats:")
print(team_penalty_stats.head())


Penalty Shootout Data:
  tournament_id            tournament_name  year      stage_name   match_id  \
0       WC-1982  1982 FIFA Men's World Cup  1982     semi-finals  M-1982-50   
1       WC-1986  1986 FIFA Men's World Cup  1986  quarter-finals  M-1986-45   
2       WC-1986  1986 FIFA Men's World Cup  1986  quarter-finals  M-1986-46   
3       WC-1986  1986 FIFA Men's World Cup  1986  quarter-finals  M-1986-48   
4       WC-1990  1990 FIFA Men's World Cup  1990     round of 16  M-1990-41   

                       match_name home_team_id       home_team_name  \
0          West Germany vs France         T-86         West Germany   
1                Brazil vs France         T-09               Brazil   
2          West Germany vs Mexico         T-86         West Germany   
3                Spain vs Belgium         T-73                Spain   
4  Republic of Ireland vs Romania         T-60  Republic of Ireland   

  away_team_id away_team_name  home_team_score_penalties  \
0         T-30 

In [12]:
print("Columns in matches:", matches.columns)
print("Columns in tournaments:", tournaments.columns)


Columns in matches: Index(['key_id', 'tournament_id', 'tournament_name', 'match_id', 'match_name',
       'stage_name', 'group_name', 'group_stage', 'knockout_stage', 'replayed',
       'replay', 'match_date', 'match_time', 'stadium_id', 'stadium_name',
       'city_name', 'country_name', 'home_team_id', 'home_team_name',
       'home_team_code', 'away_team_id', 'away_team_name', 'away_team_code',
       'score', 'home_team_score', 'away_team_score', 'home_team_score_margin',
       'away_team_score_margin', 'extra_time', 'penalty_shootout',
       'score_penalties', 'home_team_score_penalties',
       'away_team_score_penalties', 'result', 'home_team_win', 'away_team_win',
       'draw'],
      dtype='object')
Columns in tournaments: Index(['key_id', 'tournament_id', 'tournament_name', 'year', 'start_date',
       'end_date', 'host_country', 'winner', 'host_won', 'count_teams',
       'group_stage', 'second_group_stage', 'final_round', 'round_of_16',
       'quarter_finals', 'semi_fin

Player Development and Emerging Stars

In [145]:
import pandas as pd

# Load players and tournaments datasets
players = pd.read_csv('players.csv')
tournaments = pd.read_csv('tournaments.csv')

# Convert birth_date and tournament dates to datetime
players['birth_date'] = pd.to_datetime(players['birth_date'], errors='coerce')
tournaments['start_date'] = pd.to_datetime(tournaments['start_date'], errors='coerce')
tournaments['end_date'] = pd.to_datetime(tournaments['end_date'], errors='coerce')

# Remove invalid dates in tournaments
tournaments = tournaments.dropna(subset=['start_date'])

# Print dataset information
print("Players dataset columns:", players.columns)
print("Players dataset preview:")
print(players.head())

print("\nTournaments dataset columns:", tournaments.columns)
print("Tournaments dataset preview:")
print(tournaments.head())


Players dataset columns: Index(['key_id', 'player_id', 'family_name', 'given_name', 'birth_date',
       'female', 'goal_keeper', 'defender', 'midfielder', 'forward',
       'count_tournaments', 'list_tournaments', 'player_wikipedia_link'],
      dtype='object')
Players dataset preview:
   key_id player_id family_name   given_name birth_date  female  goal_keeper  \
0       1   P-35894     A'Court         Alan 1934-09-30       0            0   
1       2   P-29915     Aarønes  Ann Kristin 1973-01-19       1            0   
2       3   P-03484    Aaronson      Brenden 2000-10-22       0            0   
3       4   P-04189   Abadzhiev       Stefan 1934-07-03       0            0   
4       5   P-03523       Abalo    Jean-Paul 1975-06-26       0            0   

   defender  midfielder  forward  count_tournaments list_tournaments  \
0         0           0        1                  1             1958   
1         0           1        1                  2       1995, 1999   
2         0    

In [146]:
# Standardize tournament_id in players to match tournaments dataset format
players['tournament_id'] = 'WC-' + players['list_tournaments'].str.split(',').str[0].str.strip()

# Re-merge players with tournaments after fixing the format
players_with_tournaments = pd.merge(
    players,
    tournaments[['tournament_id', 'tournament_name', 'year', 'start_date', 'host_country']],
    how='left',
    on='tournament_id'
)

# Recalculate age at the tournament
players_with_tournaments['age_at_tournament'] = (
    players_with_tournaments['start_date'] - players_with_tournaments['birth_date']
).dt.days // 365

# Display the updated dataset
print("Updated Players with tournaments dataset columns:", players_with_tournaments.columns)
print("Updated Players with tournaments dataset preview:")
print(players_with_tournaments.head())



Updated Players with tournaments dataset columns: Index(['key_id', 'player_id', 'family_name', 'given_name', 'birth_date',
       'female', 'goal_keeper', 'defender', 'midfielder', 'forward',
       'count_tournaments', 'list_tournaments', 'player_wikipedia_link',
       'tournament_id', 'tournament_name', 'year', 'start_date',
       'host_country', 'age_at_tournament'],
      dtype='object')
Updated Players with tournaments dataset preview:
   key_id player_id family_name   given_name birth_date  female  goal_keeper  \
0       1   P-35894     A'Court         Alan 1934-09-30       0            0   
1       2   P-29915     Aarønes  Ann Kristin 1973-01-19       1            0   
2       3   P-03484    Aaronson      Brenden 2000-10-22       0            0   
3       4   P-04189   Abadzhiev       Stefan 1934-07-03       0            0   
4       5   P-03523       Abalo    Jean-Paul 1975-06-26       0            0   

   defender  midfielder  forward  count_tournaments list_tournaments  \


In [147]:
# Step 1: Group goals data by player and tournament
player_goals = goals.groupby(['player_id', 'tournament_id']).agg({
    'goal_id': 'count',  # Total goals scored
    'penalty': 'sum',    # Penalty goals
    'own_goal': 'sum'    # Own goals
}).reset_index().rename(columns={'goal_id': 'total_goals'})

# Step 2: Merge goals data with players_with_tournaments
players_with_goals = pd.merge(
    players_with_tournaments,
    player_goals,
    on=['player_id', 'tournament_id'],
    how='left'
)

# Fill missing values for players who didn't score
players_with_goals[['total_goals', 'penalty', 'own_goal']] = players_with_goals[
    ['total_goals', 'penalty', 'own_goal']
].fillna(0)

# Display the updated dataset
print("Players with goals dataset columns:", players_with_goals.columns)
print("Players with goals dataset preview:")
print(players_with_goals.head())


Players with goals dataset columns: Index(['key_id', 'player_id', 'family_name', 'given_name', 'birth_date',
       'female', 'goal_keeper', 'defender', 'midfielder', 'forward',
       'count_tournaments', 'list_tournaments', 'player_wikipedia_link',
       'tournament_id', 'tournament_name', 'year', 'start_date',
       'host_country', 'age_at_tournament', 'total_goals', 'penalty',
       'own_goal'],
      dtype='object')
Players with goals dataset preview:
   key_id player_id family_name   given_name birth_date  female  goal_keeper  \
0       1   P-35894     A'Court         Alan 1934-09-30       0            0   
1       2   P-29915     Aarønes  Ann Kristin 1973-01-19       1            0   
2       3   P-03484    Aaronson      Brenden 2000-10-22       0            0   
3       4   P-04189   Abadzhiev       Stefan 1934-07-03       0            0   
4       5   P-03523       Abalo    Jean-Paul 1975-06-26       0            0   

   defender  midfielder  forward  ...  \
0         0   

In [148]:
# Step 1: Group player appearances data
player_appearance_summary = player_appearances.groupby(
    ['player_id', 'tournament_id']
).agg({
    'match_id': 'count',  # Total matches played
    'starter': 'sum',     # Count of matches started
    'substitute': 'sum',  # Count of substitute appearances
    'position_name': lambda x: ', '.join(x.unique())  # Unique positions played
}).reset_index().rename(columns={
    'match_id': 'matches_played',
    'starter': 'matches_started',
    'substitute': 'matches_as_substitute',
    'position_name': 'positions_played'
})

# Step 2: Merge appearances data with players_with_goals
players_with_appearances = pd.merge(
    players_with_goals,
    player_appearance_summary,
    on=['player_id', 'tournament_id'],
    how='left'
)

# Fill missing values for players without appearances
players_with_appearances[['matches_played', 'matches_started', 'matches_as_substitute']] = players_with_appearances[
    ['matches_played', 'matches_started', 'matches_as_substitute']
].fillna(0)
players_with_appearances['positions_played'] = players_with_appearances['positions_played'].fillna('Unknown')

# Display the updated dataset
print("Players with appearances dataset columns:", players_with_appearances.columns)
print("Players with appearances dataset preview:")
print(players_with_appearances.head())


Players with appearances dataset columns: Index(['key_id', 'player_id', 'family_name', 'given_name', 'birth_date',
       'female', 'goal_keeper', 'defender', 'midfielder', 'forward',
       'count_tournaments', 'list_tournaments', 'player_wikipedia_link',
       'tournament_id', 'tournament_name', 'year', 'start_date',
       'host_country', 'age_at_tournament', 'total_goals', 'penalty',
       'own_goal', 'matches_played', 'matches_started',
       'matches_as_substitute', 'positions_played'],
      dtype='object')
Players with appearances dataset preview:
   key_id player_id family_name   given_name birth_date  female  goal_keeper  \
0       1   P-35894     A'Court         Alan 1934-09-30       0            0   
1       2   P-29915     Aarønes  Ann Kristin 1973-01-19       1            0   
2       3   P-03484    Aaronson      Brenden 2000-10-22       0            0   
3       4   P-04189   Abadzhiev       Stefan 1934-07-03       0            0   
4       5   P-03523       Abalo    

In [149]:
# Step 0: Add `team_id` to `players_with_appearances`
# Load squads dataset
squads = pd.read_csv('squads.csv')

# Merge `team_id` into `players_with_appearances`
players_with_appearances = pd.merge(
    players_with_appearances,
    squads[['player_id', 'team_id']].drop_duplicates(),
    on='player_id',
    how='left'
)

# Verify if `team_id` is now present
print("Players dataset with team_id columns:", players_with_appearances.columns)
print("Players dataset with team_id preview:")
print(players_with_appearances.head())

# Step 1: Group team performance data 
team_performance = group_standings.groupby(['team_id', 'tournament_id']).agg({
    'played': 'sum',
    'wins': 'sum',
    'draws': 'sum',
    'losses': 'sum',
    'goals_for': 'sum',
    'goals_against': 'sum',
    'goal_difference': 'sum',
    'points': 'sum',
    'position': 'min'  # Minimum position in the group
}).reset_index()

# Step 2: Merge team performance data with players_with_appearances
players_with_team_performance = pd.merge(
    players_with_appearances,
    team_performance,
    on=['tournament_id', 'team_id'],
    how='left'
)

# Fill missing values for players whose teams are not in group standings
players_with_team_performance.fillna({
    'played': 0, 'wins': 0, 'draws': 0, 'losses': 0,
    'goals_for': 0, 'goals_against': 0, 'goal_difference': 0,
    'points': 0, 'position': float('inf')  # Assign high value to indicate unranked
}, inplace=True)

# Display the updated dataset
print("Players with team performance dataset columns:", players_with_team_performance.columns)
print("Players with team performance dataset preview:")
print(players_with_team_performance.head())


Players dataset with team_id columns: Index(['key_id', 'player_id', 'family_name', 'given_name', 'birth_date',
       'female', 'goal_keeper', 'defender', 'midfielder', 'forward',
       'count_tournaments', 'list_tournaments', 'player_wikipedia_link',
       'tournament_id', 'tournament_name', 'year', 'start_date',
       'host_country', 'age_at_tournament', 'total_goals', 'penalty',
       'own_goal', 'matches_played', 'matches_started',
       'matches_as_substitute', 'positions_played', 'team_id'],
      dtype='object')
Players dataset with team_id preview:
   key_id player_id family_name   given_name birth_date  female  goal_keeper  \
0       1   P-35894     A'Court         Alan 1934-09-30       0            0   
1       2   P-29915     Aarønes  Ann Kristin 1973-01-19       1            0   
2       3   P-03484    Aaronson      Brenden 2000-10-22       0            0   
3       4   P-04189   Abadzhiev       Stefan 1934-07-03       0            0   
4       5   P-03523       Abalo 

In [150]:
# Step 3: Add derived metrics for team performance
players_with_team_performance['win_rate'] = (
    players_with_team_performance['wins'] / players_with_team_performance['played']
).fillna(0)

players_with_team_performance['avg_goals_per_match'] = (
    players_with_team_performance['goals_for'] / players_with_team_performance['played']
).fillna(0)

players_with_team_performance['avg_conceded_per_match'] = (
    players_with_team_performance['goals_against'] / players_with_team_performance['played']
).fillna(0)

# Step 4: Save the final dataset
players_with_team_performance.to_csv('enhanced_players_with_team_performance.csv', index=False)

# Display a preview of the enhanced dataset
print("Enhanced Players with Team Performance Dataset saved as 'enhanced_players_with_team_performance.csv'.")
print("Enhanced dataset preview:")
print(players_with_team_performance.head())


Enhanced Players with Team Performance Dataset saved as 'enhanced_players_with_team_performance.csv'.
Enhanced dataset preview:
   key_id player_id family_name   given_name birth_date  female  goal_keeper  \
0       1   P-35894     A'Court         Alan 1934-09-30       0            0   
1       2   P-29915     Aarønes  Ann Kristin 1973-01-19       1            0   
2       3   P-03484    Aaronson      Brenden 2000-10-22       0            0   
3       4   P-04189   Abadzhiev       Stefan 1934-07-03       0            0   
4       5   P-03523       Abalo    Jean-Paul 1975-06-26       0            0   

   defender  midfielder  forward  ...  draws losses goals_for goals_against  \
0         0           0        1  ...    3.0    0.0       4.0           4.0   
1         0           1        1  ...    0.0    0.0      17.0           0.0   
2         0           0        1  ...    2.0    0.0       2.0           1.0   
3         0           0        1  ...    0.0    3.0       1.0           8.0

National Team Progression and Growth

In [154]:
import pandas as pd

# Load datasets
matches = pd.read_csv('matches.csv')
tournaments = pd.read_csv('tournaments.csv')
group_standings = pd.read_csv('group_standings.csv')
qualified_teams = pd.read_csv('qualified_teams.csv')

# Display dataset previews
print("Matches dataset columns:", matches.columns)
print(matches.head())

print("Tournaments dataset columns:", tournaments.columns)
print(tournaments.head())

print("Group standings dataset columns:", group_standings.columns)
print(group_standings.head())

print("Qualified teams dataset columns:", qualified_teams.columns)
print(qualified_teams.head())

# Step 1: Merge matches with tournaments to include year and host country
matches_with_tournaments = pd.merge(
    matches,
    tournaments[['tournament_id', 'year', 'host_country']],
    on='tournament_id',
    how='left'
)

# Step 2: Melt the matches dataset to create a long format for team-level analysis
matches_long = pd.melt(
    matches_with_tournaments,
    id_vars=[
        'match_id',              # Match identifier
        'tournament_id',         # Tournament identifier
        'tournament_name',       # Tournament name
        'year',                  # Year of the tournament
        'host_country'           # Host country of the tournament
    ],
    value_vars=['home_team_id', 'away_team_id'],  # Home and away team IDs
    var_name='team_type',        # Variable name for team type (home/away)
    value_name='team_id'         # Variable name for team ID
)

# Display the reshaped dataset
print("Matches long dataset columns:", matches_long.columns)
print(matches_long.head())

# Step 3: Aggregate match-level data by team_id
team_progression = matches_long.groupby(['team_id', 'tournament_id']).agg({
    'match_id': 'count',  # Total matches played
    'team_type': lambda x: (x == 'home_team_id').sum(),  # Home matches (proxy for wins)
}).reset_index()

# Rename columns for clarity
team_progression.rename(columns={
    'match_id': 'total_matches',
    'team_type': 'home_matches'
}, inplace=True)

# Step 4: Add performance labels from the qualified_teams dataset
team_progression = pd.merge(
    team_progression,
    qualified_teams[['tournament_id', 'team_id', 'performance']],
    on=['tournament_id', 'team_id'],
    how='left'
)

# Step 5: Include year and host details from the tournaments dataset
team_progression = pd.merge(
    team_progression,
    tournaments[['tournament_id', 'year', 'host_country']],
    on='tournament_id',
    how='left'
)

# Step 6: Calculate win percentage for home matches
team_progression['home_win_percentage'] = (
    team_progression['home_matches'] / team_progression['total_matches']
) * 100

# Step 7: Assign numeric scores to performance levels
performance_mapping = {
    'winner': 1,
    'runner-up': 2,
    'semi-finals': 3,
    'quarter-finals': 4,
    'round of 16': 5,
    'group stage': 6
}
team_progression['performance_score'] = team_progression['performance'].map(performance_mapping)

# Step 8: Count number of tournaments per team
team_progression['tournaments_participated'] = team_progression.groupby('team_id')['tournament_id'].transform('count')

# Display the enhanced dataset
print("Enhanced Team Progression dataset columns:", team_progression.columns)
print(team_progression.head())

# Step 9: Save the enhanced dataset to a CSV file for Tableau
team_progression.to_csv("enhanced_team_progression.csv", index=False)
print("Enhanced Team Progression dataset saved as 'enhanced_team_progression.csv'.")


Matches dataset columns: Index(['key_id', 'tournament_id', 'tournament_name', 'match_id', 'match_name',
       'stage_name', 'group_name', 'group_stage', 'knockout_stage', 'replayed',
       'replay', 'match_date', 'match_time', 'stadium_id', 'stadium_name',
       'city_name', 'country_name', 'home_team_id', 'home_team_name',
       'home_team_code', 'away_team_id', 'away_team_name', 'away_team_code',
       'score', 'home_team_score', 'away_team_score', 'home_team_score_margin',
       'away_team_score_margin', 'extra_time', 'penalty_shootout',
       'score_penalties', 'home_team_score_penalties',
       'away_team_score_penalties', 'result', 'home_team_win', 'away_team_win',
       'draw'],
      dtype='object')
   key_id tournament_id            tournament_name   match_id  \
0       1       WC-1930  1930 FIFA Men's World Cup  M-1930-01   
1       2       WC-1930  1930 FIFA Men's World Cup  M-1930-02   
2       3       WC-1930  1930 FIFA Men's World Cup  M-1930-03   
3       4     

In [26]:
import pandas as pd

# Load datasets
tournaments = pd.read_csv("tournaments.csv")
matches = pd.read_csv("matches.csv")
qualified_teams = pd.read_csv("qualified_teams.csv")
team_appearances = pd.read_csv("team_appearances.csv")
player_appearances = pd.read_csv("player_appearances.csv")
squads = pd.read_csv("squads.csv")
goals = pd.read_csv("goals.csv")
group_standings = pd.read_csv("group_standings.csv")
stadiums = pd.read_csv("stadiums.csv")

# Step 1: Base Dataset from Matches
matches_long = pd.melt(
    matches,
    id_vars=['match_id', 'tournament_id', 'match_date', 'stage_name', 'stadium_id'],
    value_vars=['home_team_id', 'away_team_id'],
    var_name='team_type',
    value_name='team_id'
)

# Step 2: Merge Tournament Data
matches_with_tournaments = pd.merge(
    matches_long,
    tournaments[['tournament_id', 'tournament_name', 'year', 'host_country']],
    on='tournament_id',
    how='left'
)

# Step 3: Merge Qualified Teams Data
matches_with_teams = pd.merge(
    matches_with_tournaments,
    qualified_teams[['tournament_id', 'team_id', 'performance']],
    on=['tournament_id', 'team_id'],
    how='left'
)

# Step 4: Add Team Appearances Data
matches_with_team_appearances = pd.merge(
    matches_with_teams,
    team_appearances[['match_id', 'team_id', 'goals_for', 'goals_against', 'win', 'lose', 'draw']],
    on=['match_id', 'team_id'],
    how='left'
)

# Step 5: Add Stadium Information
matches_with_stadiums = pd.merge(
    matches_with_team_appearances,
    stadiums[['stadium_id', 'stadium_name', 'city_name', 'country_name']],
    on='stadium_id',
    how='left'
)

# Step 6: Add Player Data from Squads and Player Appearances
players_with_matches = pd.merge(
    player_appearances,
    squads[['player_id', 'team_id']].drop_duplicates(),
    on='player_id',
    how='left'
)

# Step 7: Add Goals Data to Players
players_with_goals = pd.merge(
    players_with_matches,
    goals[['player_id', 'match_id', 'goal_id']].groupby(['player_id', 'match_id']).count().reset_index(),
    on=['player_id', 'match_id'],
    how='left'
).rename(columns={'goal_id': 'goals_scored'})

# Ensure 'team_id' exists in players_with_goals
if 'team_id' not in players_with_goals.columns:
    players_with_goals = pd.merge(
        players_with_goals,
        squads[['player_id', 'team_id']].drop_duplicates(),
        on='player_id',
        how='left'
    )

# Step 8: Merge Player-Level Data into Matches
master_dataset = pd.merge(
    matches_with_stadiums,
    players_with_goals,
    on=['match_id', 'team_id'],
    how='left'
)

# Step 9: Rename Columns to Prevent Conflicts
master_dataset = master_dataset.rename(columns={'team_id': 'team_id_master'})
group_standings = group_standings.rename(columns={'team_id': 'team_id_standings'})

# Debugging: Check columns in both dataframes
print("Columns in master_dataset before merging group_standings:", master_dataset.columns)
print("Columns in group_standings before merging:", group_standings.columns)

# Ensure the columns exist in both dataframes
if 'tournament_id' not in master_dataset.columns:
    raise KeyError("'tournament_id' is missing in master_dataset.")
if 'tournament_id' not in group_standings.columns:
    raise KeyError("'tournament_id' is missing in group_standings.")

# Merge Group Standings into Master Dataset
master_dataset = pd.merge(
    master_dataset,
    group_standings[['tournament_id', 'team_id_standings', 'position', 'points', 'goal_difference']],
    left_on=['tournament_id', 'team_id_master'],
    right_on=['tournament_id', 'team_id_standings'],
    how='left'
)

# Debugging: Confirm merge success
print("Columns in master_dataset after merging group_standings:", master_dataset.columns)

# Step 11: Organize Columns
master_dataset = master_dataset[[
    'tournament_id', 'tournament_name', 'year', 'host_country',
    'match_id', 'match_date', 'stage_name', 'stadium_name', 'city_name', 'country_name',
    'team_id_master', 'team_type', 'performance', 'goals_for', 'goals_against', 'win', 'lose', 'draw',
    'player_id', 'goals_scored', 'position', 'points', 'goal_difference'
]]

# Step 12: Save Master Dataset
master_dataset.to_csv('master_linking_dataset.csv', index=False)

# Display Final Dataset
print("Master Linking Dataset Preview:")
print(master_dataset.head())
print(f"Dataset Shape: {master_dataset.shape}")


Columns in master_dataset before merging group_standings: Index(['match_id', 'tournament_id_x', 'match_date_x', 'stage_name_x',
       'stadium_id', 'team_type', 'team_id_master', 'tournament_name_x',
       'year', 'host_country', 'performance', 'goals_for', 'goals_against',
       'win', 'lose', 'draw', 'stadium_name', 'city_name', 'country_name',
       'key_id', 'tournament_id_y', 'tournament_name_y', 'match_name',
       'match_date_y', 'stage_name_y', 'group_name', 'team_id_x', 'team_name',
       'team_code', 'home_team', 'away_team', 'player_id', 'family_name',
       'given_name', 'shirt_number', 'position_name', 'position_code',
       'starter', 'substitute', 'team_id_y', 'goals_scored'],
      dtype='object')
Columns in group_standings before merging: Index(['key_id', 'tournament_id', 'tournament_name', 'stage_number',
       'stage_name', 'group_name', 'position', 'team_id_standings',
       'team_name', 'team_code', 'played', 'wins', 'draws', 'losses',
       'goals_for',

KeyError: "'tournament_id' is missing in master_dataset."

In [ ]:
print("Columns in master_dataset:", master_dataset.columns)
print("Columns in group_standings:", group_standings.columns)
print("Columns in master_dataset:", master_dataset.columns)
print("Columns in group_standings:", group_standings.columns)


Columns in master_dataset: Index(['match_id', 'tournament_id_x', 'match_date_x', 'stage_name_x',
       'stadium_id', 'team_type', 'team_id', 'tournament_name_x', 'year',
       'host_country', 'performance', 'goals_for', 'goals_against', 'win',
       'lose', 'draw', 'stadium_name', 'city_name', 'country_name', 'key_id',
       'tournament_id_y', 'tournament_name_y', 'match_name', 'match_date_y',
       'stage_name_y', 'group_name', 'team_id_x', 'team_name', 'team_code',
       'home_team', 'away_team', 'player_id', 'family_name', 'given_name',
       'shirt_number', 'position_name', 'position_code', 'starter',
       'substitute', 'team_id_y', 'goals_scored'],
      dtype='object')
Columns in group_standings: Index(['key_id', 'tournament_id', 'tournament_name', 'stage_number',
       'stage_name', 'group_name', 'position', 'team_id', 'team_name',
       'team_code', 'played', 'wins', 'draws', 'losses', 'goals_for',
       'goals_against', 'goal_difference', 'points', 'advanced'],
   